In [1]:
import pandas as pd
from tqdm.auto import tqdm


full_articles = pd.read_csv("articles_full.csv")
print("Number of articles before cleaning: {}".format(len(full_articles)))
full_articles = full_articles[full_articles['full_article'].notna()]
print("Number of articles after cleaning: {}".format(len(full_articles)))
full_articles["LLM_Output"] = full_articles["LLM_Output"].astype(str)

Number of articles before cleaning: 17044
Number of articles after cleaning: 16769


In [2]:
from openai import OpenAI
client = OpenAI(api_key="{YOUR_KEY}")

In [ ]:

message = """
Consider you are providing a TLDR of a finance news article for a project.
The summary needs to be valuable to build a sentiment analysis model on it.
If the article is already less than 5 lines, then provide a 2 line summary, else always provide 5 line summary.
I will also provide the company name for which the article was extracted.
Following should be the format of your output.
Try to make the summary with respect to the company name I am providing.
The sentiment should be with respect the company name I am providing.
If you are not confident on the sentiment, put "Unsure".
For relevance, based on the article provide if the article is high or low relevant. If not relevant put NA.

Summary: <Summary of the article in 5 lines>
Overall sentiment: <Positive or Negative>
Relevance to company: <High, Low or NA>
Company Name: <Repeat the company I am providing>
"""
for index, row in tqdm(full_articles.iterrows(), total=full_articles.shape[0]):
    if row["LLM_Output"] != "0":
        continue
    article = row['full_article']
    holding = row['holding']
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {
          "role": "system",
          "content": message
        },
        {
          "role": "user",
          "content": "company name: {} \n{}".format(holding, article)
        }
      ]
    )
    full_articles.at[index,'LLM_Output'] = response.choices[0].message.content

  0%|          | 0/16769 [00:00<?, ?it/s]